In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import matplotlib.pyplot as plt
import time
import copy
import scipy.integrate as integrate
from __future__ import unicode_literals
from sklearn import preprocessing as prep
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.mixture import GaussianMixture as gm
from multiprocessing import Pool
from sklearn.cluster import KMeans

In [4]:
class LOB:
    def __init__(self, wiersz):
        self.zmiana = 0
        self.dzien = int(wiersz[0]) % 100
        self.czas = int(wiersz[1])
        i = 3
        while (wiersz[i] != 'ASK'):
            i += 1
        bid = wiersz[3:i]
        ask = wiersz[(i+1):]
        self.bid = np.zeros((int((len(bid) / 2)), 2))
        self.ask = np.zeros((int((len(ask) / 2)), 2))
        for i in range(0, len(bid), 2):
            self.bid[int(i / 2)][0] = float(bid[i]) 
            self.bid[int(i / 2)][1] = float(bid[i+1])
        for i in range(0, len(ask), 2):
            self.ask[int(i / 2)] = [float(ask[i]), float(ask[i+1])]
        self.midPrice = (self.bid[len(self.bid) - 1][0] + self.ask[0][0]) / 2
    def __str__(self):
        return "Dzień: " + self.dzien + ", Czas: " + self.czas + ", midPrice: " + str(self.midPrice)

def generujZmiane(X):
    for i in range(len(X) - 1):
        if (X[i].midPrice < X[i + 1].midPrice):
            X[i].zmiana = 1
        if (X[i].midPrice > X[i + 1].midPrice):
            X[i].zmiana = -1
        if (X[i].midPrice == X[i + 1].midPrice):
            X[i].zmiana = 0
    return np.array([i.zmiana for i in X])

def czas(h, m):
    mnoznikM = 100000
    mnoznikH = 10000000
    return h*mnoznikH + m*mnoznikM

def czasOdwrotny(x):
    mnoznikM = 100000
    mnoznikH = 10000000
    return (int(x / mnoznikH), int((x % mnoznikH) / mnoznikM))

def generujPozycje(x):
    pozLewa = (x.bid[-1][0] - minimum ) * 10
    pozPrawa = (x.ask[0][0] - minimum ) * 10
    pozLewaInt = int(np.round(pozLewa))
    pozPrawaInt = int(np.round(pozPrawa))
    return (pozLewaInt, pozPrawaInt)

def generujPrzedzialy(x, ilosc, f):
    pozLewa, pozPrawa = generujPozycje(x)
    przedzialy = []
    wspXowe = []
    for i in range(ilosc):
        fi = f(i)
        wspXowe.append((pozPrawa / 10) + minimum + (fi / 2))
        wspXowe.append((pozLewa / 10) + minimum - (fi / 2))
        fi10 = int(np.floor(fi*10))
        tmpList = np.arange(fi10)
        przedzialy.append(pozPrawa + tmpList)
        pozPrawa += fi10
        pozLewa -= fi10
        przedzialy.append(pozLewa + 1 + tmpList)
    return przedzialy, wspXowe
        
def generujTabliceTickow(tablica, X, ilosc, f):
    wynik = np.zeros((len(X), 2 * ilosc))
    listaWspXowych = []
    for i in range(len(X)):
        wiersz = tablica[i]
        przedzialy, wspXowe = generujPrzedzialy(X[i], ilosc, f)
        listaWspXowych.append(wspXowe)
        for j in range(2 * ilosc):
            przedzial = przedzialy[j]
            przedzial = przedzial[przedzial >= 0]
            przedzial = przedzial[przedzial < liczbaWspolrzednych]
            wynik[i][j] = np.sum(wiersz[przedzial])
    return wynik, listaWspXowych

def narysujPrzykladowyWykres(xAsk, yAsk, xBid, yBid, midPrice, nawa, sizex = 8, sizey = 4, zblizone = False):
    plt.rc('font', family='Arial')
    plt.rcParams["figure.figsize"] = [sizex, sizey]
    maks = np.max([np.max(yBid), np.max(yAsk)])
    plt.xlabel('Cena')
    plt.ylabel("Ilość")
    szerokosc = 0.2
    if (zblizone):
        plt.xticks(np.arange(midPrice - 12, midPrice + 12, 2))
        lewe = midPrice - 8
        prawe = midPrice + 8
        maks = np.max([np.max(yBid[xBid >= lewe]), np.max(yAsk[xAsk <= prawe])])
        plt.axis([lewe, prawe, 0, maks])
        szerokosc /= 10
    else:
        plt.xticks(np.arange(midPrice-75, midPrice+125, 25))
        plt.axis([midPrice-75, midPrice+100, 0, maks])
    plt.title("Ilość akcji w stosunku do ceny")
    
    plt.bar(xBid, yBid, width=szerokosc, color = 'blue', edgecolor = "blue", label = "BID")
    plt.bar(xAsk, yAsk, width=szerokosc, color = 'red', edgecolor = "red", label = "ASK")
    
    plt.bar(midPrice, maks, width=szerokosc, color = 'green', edgecolor = "green", label = "mid-price")

    plt.legend()

    plt.tight_layout()
    plt.savefig(nawa)
    plt.show()

def generujImbalance(X):
    i = 0
    Imbalance = np.zeros((X.shape[0], 1))
    for wiersz in X: 
        bid = wiersz.bid[-1, 1]
        ask = wiersz.ask[0, 1]
        Imbalance[i] = (bid - ask) / (bid + ask)
        i += 1
    return Imbalance

def standaryzowanie(tab):
    tabS = prep.StandardScaler().fit_transform(tab)
    tabN = prep.Normalizer().fit_transform(tab)
    tabSN = prep.Normalizer().fit_transform(tabS)
    tabNS = prep.StandardScaler().fit_transform(tabN)
    return tab, tabS, tabN, tabSN, tabNS

def zwrocDzien(x):
    return {
        0 : 'poniedziałek',
        1 : 'wtorek',
        2 : 'środa',
        3 : 'czwartek',
        4 : 'piątek'
    }[x % 5]

def test2( nazwa, tablica, tablica2, tablica3, tablica4, tablica5, svm, ilosc = 4, procent = 0.75, iloscKolumn = 200):
    wynik1 = crossValidationPoKolei(svm, (((tablica).T[:iloscKolumn]).T), zmiana, ilosc, procent)
    wynik2 = crossValidationPoKolei(svm, (((tablica2).T[:iloscKolumn]).T), zmiana, ilosc, procent)
    wynik3 = crossValidationPoKolei(svm, (((tablica3).T[:iloscKolumn]).T), zmiana, ilosc, procent)
    wynik4= crossValidationPoKolei(svm, (((tablica4).T[:iloscKolumn]).T), zmiana, ilosc, procent) 
    wynik5 = crossValidationPoKolei(svm, (((tablica5).T[:iloscKolumn]).T), zmiana, ilosc, procent) 
    return wynik1, wynik2, wynik3, wynik4, wynik5


def wypisz(TAB, liczbaKolumn = 200):
    tab, tabS, tabN, tabSN, tabNS = standaryzowanie(TAB)
    gammas = [1e-6, 1e-4, 1e-2, 1, 1e2, 1e4, 1e6]
    Cs = [1e-6, 1e-4, 1e-2, 1, 1e2, 1e4, 1e6]

    procent = 0.7
    print("procent =", procent)
    print('Czyste  S     N     S->N    N->S')
    for gamma in gammas:
        for C in Cs:
            svm = SVC(class_weight='balanced', gamma = gamma, kernel = 'rbf', C = C)
            wynik1, wynik2, wynik3, wynik4, wynik5 = test2("imbalance", tab, tabS, tabN, tabSN, tabNS, svm, ilosc = 4, procent = procent, iloscKolumn = liczbaKolumn)
            odpowiedz = "{0:0.4f}  {1:0.4f}  {2:0.4f}  {3:0.4f}  {4:0.4f}".format(wynik1, wynik2, wynik3, wynik4, wynik5)
            if (wynik1 > 0.52 or wynik2 > 0.52 or wynik3 > 0.52 or wynik4 > 0.52 or wynik5 > 0.52):
                odpowiedz = "\x1b[31m\"{0:0.4f}  {1:0.4f}  {2:0.4f}  {3:0.4f}  {4:0.4f}\"\x1b[0m".format(wynik1, wynik2, wynik3, wynik4, wynik5)

            print(odpowiedz, "- dla: g:", gamma, "C:", C) 
                

def testujDecisionFunction(xTest, yTest, najDecFun, clf):
    wynik = clf.decision_function(xTest).reshape(yTest.shape)
    yTestWiekszeOdZera = yTest > 0
    if (najDecFun[1]):
        return np.sum((wynik > najDecFun[0]) == yTestWiekszeOdZera) / yTest.shape[0]
    return np.sum((wynik < najDecFun[0]) == yTestWiekszeOdZera) / yTest.shape[0]

def najlepszeDecFun(yTrain, wynikDecFun):
    yTrainWiekszeOdZera = yTrain > 0
    tabWynikow = []
    for i in range(wynikDecFun.shape[0]):
        wy = np.sum((wynikDecFun < wynikDecFun[i]) == yTrainWiekszeOdZera) / yTrain.shape[0]
        tabWynikow.append(wy)
        wy = np.sum((wynikDecFun > wynikDecFun[i]) == yTrainWiekszeOdZera) / yTrain.shape[0]
        tabWynikow.append(wy)
    pozycjaMaxArg = np.argmax(tabWynikow)
    if (pozycjaMaxArg % 2 == 0):
        return (wynikDecFun[int(pozycjaMaxArg / 2)], 0)
    else:
        return (wynikDecFun[int(pozycjaMaxArg / 2)], 1)

def crossValidation(clf, data, dataClass, k):
    size = data.shape[0]
    arr = np.arange(size)
    np.random.shuffle(arr)
    err = 0
    n = int(size / k)
    for i in range(k):
        mask2 = arr[np.arange(i*n,min((i+1)*n,size))]
        mask1 = arr[np.concatenate((np.arange(i*n),np.arange((i+1)*n,size)))]
        X_train = data[mask1]
        y_train = dataClass[mask1]
        X_test = data[mask2]
        y_test = dataClass[mask2]  
        clf = clf.fit(X_train, y_train)
        y_train = y_train.reshape((y_train.shape[0], 1))
        wynikDecFun = clf.decision_function(X_train).reshape(y_train.shape)
        najDecFun = najlepszeDecFun(y_train, wynikDecFun)        
        err += testujDecisionFunction(X_test, y_test, najDecFun, clf)
    return float(err) / k

def crossValidationPoKolei(clf, data, dataClass, k, p):
    size = data.shape[0]
    arr = np.arange(size)
    err = 0
    n = int(size / (k+1))
    trainCount = int(2 * n * p)
    for i in range(k):
        trainMask = np.arange(i*n, i*n + trainCount)
        testMask = np.arange(i*n + trainCount, (i+2)*n)
        X_train = data[trainMask]
        y_train = dataClass[trainMask]
        X_test = data[testMask]
        y_test = dataClass[testMask]  
        clf = clf.fit(X_train, y_train)
#         y_train = y_train.reshape((y_train.shape[0], 1))
#         wynikDecFun = clf.decision_function(X_train).reshape(y_train.shape)
#         najDecFun = najlepszeDecFun(y_train, wynikDecFun)        
#         wynik = testujDecisionFunction(X_test, y_test, najDecFun, clf)
#         err += wynik
        err += clf.score(X_test, y_test)
    return float(err) / k

In [2]:
tablicaTickowPo1 = np.load("tablicaTickowPo1.npy")
tablicaTickowPo1M = np.load("tablicaTickowPo1M.npy")
tablicaTickowLogarytmiczna = np.load("tablicaTickowLogarytmiczna.npy")
tablicaTickowLogarytmicznaM = np.load("tablicaTickowLogarytmicznaM.npy")
tablicaTickowLogarytmiczna2 = np.load("tablicaTickowLogarytmiczna2.npy")
tablicaTickowLogarytmiczna2M = np.load("tablicaTickowLogarytmiczna2M.npy", )
tablicaTickowRosnaca = np.load("tablicaTickowRosnaca.npy")
tablicaTickowRosnacaM = np.load("tablicaTickowRosnacaM.npy")
tablica = np.load("tablica.npy")
zmiana = np.load("zmiana.npy")

In [5]:
wypisz(tablicaTickowRosnaca)

procent = 0.7
Czyste  S     N     S->N    N->S
0.5032  0.5032  0.5032  0.5032  0.5032 - dla: g: 1e-06 C: 1e-06
0.4909  0.4909  0.4909  0.4909  0.4909 - dla: g: 1e-06 C: 0.0001
0.5100  0.5100  0.5100  0.5100  0.5100 - dla: g: 1e-06 C: 0.01
0.4900  0.4778  0.4778  0.4778  0.4778 - dla: g: 1e-06 C: 1
0.4900  0.4909  0.4909  0.4909  0.4909 - dla: g: 1e-06 C: 100.0
0.4900  0.4982  0.5032  0.5050  0.4923 - dla: g: 1e-06 C: 10000.0
0.4900  0.4932  0.4950  0.4986  0.4932 - dla: g: 1e-06 C: 1000000.0
0.5032  0.5032  0.5032  0.5032  0.5032 - dla: g: 0.0001 C: 1e-06
0.4909  0.4909  0.4909  0.4909  0.4909 - dla: g: 0.0001 C: 0.0001
0.5100  0.5100  0.5100  0.5100  0.5100 - dla: g: 0.0001 C: 0.01
0.4900  0.4778  0.4778  0.4778  0.4778 - dla: g: 0.0001 C: 1
0.4900  0.5014  0.4909  0.4923  0.4959 - dla: g: 0.0001 C: 100.0
0.4900  0.4900  0.4964  0.4959  0.4859 - dla: g: 0.0001 C: 10000.0
0.4900  0.4941  0.4873  0.4900  0.5045 - dla: g: 0.0001 C: 1000000.0
0.5032  0.5032  0.5032  0.5032  0.5032 - dla: 

In [17]:
wypisz(np.abs(tablicaTickowLogarytmiczna2))

procent = 0.7
Czyste  S     N     S->N    N->S
0.5032  0.5032  0.5032  0.5032  0.5032 - dla: g: 1e-06 C: 1e-06
0.4909  0.4909  0.4909  0.4909  0.4909 - dla: g: 1e-06 C: 0.0001
0.5100  0.5100  0.5100  0.5100  0.5100 - dla: g: 1e-06 C: 0.01
0.4900  0.4778  0.4778  0.4778  0.4778 - dla: g: 1e-06 C: 1
0.4900  0.4968  0.4909  0.4909  0.5023 - dla: g: 1e-06 C: 100.0
0.4900  0.5113  0.5032  0.5032  0.5136 - dla: g: 1e-06 C: 10000.0
0.4900  0.5068  0.5100  0.5054  0.4905 - dla: g: 1e-06 C: 1000000.0
0.5032  0.5032  0.5032  0.5032  0.5032 - dla: g: 0.0001 C: 1e-06
0.4909  0.4909  0.4909  0.4909  0.4909 - dla: g: 0.0001 C: 0.0001
0.5100  0.5100  0.5100  0.5100  0.5100 - dla: g: 0.0001 C: 0.01
0.4900  0.4923  0.4778  0.4778  0.4973 - dla: g: 0.0001 C: 1
0.4900  0.5145  0.4909  0.4909  0.5177 - dla: g: 0.0001 C: 100.0
0.4900  0.4973  0.5050  0.5032  0.4991 - dla: g: 0.0001 C: 10000.0
0.4900  0.5005  0.4991  0.5041  0.5018 - dla: g: 0.0001 C: 1000000.0
0.5032  0.5032  0.5032  0.5032  0.5032 - dla: 

In [6]:
wypisz(tablicaTickowLogarytmiczna2)

procent = 0.7
Czyste  S     N     S->N    N->S
0.5032  0.5032  0.5032  0.5032  0.5032 - dla: g: 1e-06 C: 1e-06
0.4909  0.4909  0.4909  0.4909  0.4909 - dla: g: 1e-06 C: 0.0001
0.5100  0.5100  0.5100  0.5100  0.5100 - dla: g: 1e-06 C: 0.01
0.4900  0.4778  0.4778  0.4778  0.4778 - dla: g: 1e-06 C: 1
0.4900  0.4968  0.4909  0.4909  0.5023 - dla: g: 1e-06 C: 100.0
0.4900  0.5113  0.5032  0.5032  0.5136 - dla: g: 1e-06 C: 10000.0
0.4900  0.5068  0.5100  0.5054  0.4905 - dla: g: 1e-06 C: 1000000.0
0.5032  0.5032  0.5032  0.5032  0.5032 - dla: g: 0.0001 C: 1e-06
0.4909  0.4909  0.4909  0.4909  0.4909 - dla: g: 0.0001 C: 0.0001
0.5100  0.5100  0.5100  0.5100  0.5100 - dla: g: 0.0001 C: 0.01
0.4900  0.4923  0.4778  0.4778  0.4973 - dla: g: 0.0001 C: 1
0.4900  0.5145  0.4909  0.4909  0.5177 - dla: g: 0.0001 C: 100.0
0.4900  0.4973  0.5050  0.5032  0.4991 - dla: g: 0.0001 C: 10000.0
0.4900  0.5005  0.4991  0.5041  0.5018 - dla: g: 0.0001 C: 1000000.0
0.5032  0.5032  0.5032  0.5032  0.5032 - dla: 

In [7]:
wypisz(tablicaTickowLogarytmiczna)

procent = 0.7
Czyste  S     N     S->N    N->S
0.5032  0.5032  0.5032  0.5032  0.5032 - dla: g: 1e-06 C: 1e-06
0.4909  0.4909  0.4909  0.4909  0.4909 - dla: g: 1e-06 C: 0.0001
0.5100  0.5100  0.5100  0.5100  0.5100 - dla: g: 1e-06 C: 0.01
0.4900  0.4778  0.4778  0.4778  0.4778 - dla: g: 1e-06 C: 1
0.4900  0.4855  0.4909  0.4909  0.4891 - dla: g: 1e-06 C: 100.0
0.4900  0.4973  0.5032  0.5032  0.4932 - dla: g: 1e-06 C: 10000.0
0.4900  0.5009  0.5027  0.4986  0.5014 - dla: g: 1e-06 C: 1000000.0
0.5032  0.5032  0.5032  0.5032  0.5032 - dla: g: 0.0001 C: 1e-06
0.4909  0.4909  0.4909  0.4909  0.4909 - dla: g: 0.0001 C: 0.0001
0.5100  0.5100  0.5100  0.5100  0.5100 - dla: g: 0.0001 C: 0.01
0.4900  0.4868  0.4778  0.4778  0.4905 - dla: g: 0.0001 C: 1
0.4900  0.4973  0.4909  0.4909  0.4918 - dla: g: 0.0001 C: 100.0
0.4900  0.5073  0.5168  0.5023  0.5054 - dla: g: 0.0001 C: 10000.0
0.4900  0.5009  0.4968  0.4896  0.4977 - dla: g: 0.0001 C: 1000000.0
0.5032  0.5032  0.5032  0.5032  0.5032 - dla: 

In [8]:
wypisz(tablicaTickowLogarytmiczna, 80)

procent = 0.7
Czyste  S     N     S->N    N->S
0.5032  0.5032  0.5032  0.5032  0.5032 - dla: g: 1e-06 C: 1e-06
0.4909  0.4909  0.4909  0.4909  0.4909 - dla: g: 1e-06 C: 0.0001
0.5100  0.5100  0.5100  0.5100  0.5100 - dla: g: 1e-06 C: 0.01
0.4900  0.4778  0.4778  0.4778  0.4778 - dla: g: 1e-06 C: 1
0.4900  0.4909  0.4909  0.4909  0.4909 - dla: g: 1e-06 C: 100.0
0.4900  0.5118  0.5032  0.5032  0.5127 - dla: g: 1e-06 C: 10000.0
0.4900  0.5100  0.5027  0.5073  0.5104 - dla: g: 1e-06 C: 1000000.0
0.5032  0.5032  0.5032  0.5032  0.5032 - dla: g: 0.0001 C: 1e-06
0.4909  0.4909  0.4909  0.4909  0.4909 - dla: g: 0.0001 C: 0.0001
0.5100  0.5100  0.5100  0.5100  0.5100 - dla: g: 0.0001 C: 0.01
0.4900  0.4778  0.4778  0.4778  0.4778 - dla: g: 0.0001 C: 1
0.4900  0.5136  0.4909  0.4909  0.5127 - dla: g: 0.0001 C: 100.0
0.4900  0.4991  0.5027  0.5077  0.5018 - dla: g: 0.0001 C: 10000.0
0.4900  0.5054  0.5100  0.5064  0.5059 - dla: g: 0.0001 C: 1000000.0
0.5032  0.5032  0.5032  0.5032  0.5032 - dla: 

In [9]:
wypisz(tablicaTickowRosnacaM)

procent = 0.7
Czyste  S     N     S->N    N->S
0.5032  0.5032  0.5032  0.5032  0.5032 - dla: g: 1e-06 C: 1e-06
0.4909  0.4909  0.4909  0.4909  0.4909 - dla: g: 1e-06 C: 0.0001
0.5100  0.5100  0.5100  0.5100  0.5100 - dla: g: 1e-06 C: 0.01
0.4900  0.4778  0.4778  0.4778  0.4778 - dla: g: 1e-06 C: 1
0.4900  0.4909  0.4909  0.4909  0.4909 - dla: g: 1e-06 C: 100.0
0.4900  0.4982  0.5032  0.5050  0.4923 - dla: g: 1e-06 C: 10000.0
0.4900  0.4932  0.4950  0.4986  0.4932 - dla: g: 1e-06 C: 1000000.0
0.5032  0.5032  0.5032  0.5032  0.5032 - dla: g: 0.0001 C: 1e-06
0.4909  0.4909  0.4909  0.4909  0.4909 - dla: g: 0.0001 C: 0.0001
0.5100  0.5100  0.5100  0.5100  0.5100 - dla: g: 0.0001 C: 0.01
0.4900  0.4778  0.4778  0.4778  0.4778 - dla: g: 0.0001 C: 1
0.4900  0.5014  0.4909  0.4923  0.4959 - dla: g: 0.0001 C: 100.0
0.4900  0.4900  0.4964  0.4959  0.4859 - dla: g: 0.0001 C: 10000.0
0.4900  0.4941  0.4873  0.4900  0.5045 - dla: g: 0.0001 C: 1000000.0
0.5032  0.5032  0.5032  0.5032  0.5032 - dla: 

In [10]:
wypisz(tablicaTickowLogarytmiczna2M)

procent = 0.7
Czyste  S     N     S->N    N->S
0.5032  0.5032  0.5032  0.5032  0.5032 - dla: g: 1e-06 C: 1e-06
0.4909  0.4909  0.4909  0.4909  0.4909 - dla: g: 1e-06 C: 0.0001
0.5100  0.5100  0.5100  0.5100  0.5100 - dla: g: 1e-06 C: 0.01
0.4900  0.4778  0.4778  0.4778  0.4778 - dla: g: 1e-06 C: 1
0.4900  0.4968  0.4909  0.4909  0.5023 - dla: g: 1e-06 C: 100.0
0.4900  0.5113  0.5032  0.5032  0.5136 - dla: g: 1e-06 C: 10000.0
0.4900  0.5068  0.5100  0.5054  0.4905 - dla: g: 1e-06 C: 1000000.0
0.5032  0.5032  0.5032  0.5032  0.5032 - dla: g: 0.0001 C: 1e-06
0.4909  0.4909  0.4909  0.4909  0.4909 - dla: g: 0.0001 C: 0.0001
0.5100  0.5100  0.5100  0.5100  0.5100 - dla: g: 0.0001 C: 0.01
0.4900  0.4923  0.4778  0.4778  0.4973 - dla: g: 0.0001 C: 1
0.4900  0.5145  0.4909  0.4909  0.5177 - dla: g: 0.0001 C: 100.0
0.4900  0.4973  0.5050  0.5032  0.4991 - dla: g: 0.0001 C: 10000.0
0.4900  0.5005  0.4991  0.5041  0.5018 - dla: g: 0.0001 C: 1000000.0
0.5032  0.5032  0.5032  0.5032  0.5032 - dla: 

In [11]:
wypisz(tablicaTickowLogarytmicznaM)

procent = 0.7
Czyste  S     N     S->N    N->S
0.5032  0.5032  0.5032  0.5032  0.5032 - dla: g: 1e-06 C: 1e-06
0.4909  0.4909  0.4909  0.4909  0.4909 - dla: g: 1e-06 C: 0.0001
0.5100  0.5100  0.5100  0.5100  0.5100 - dla: g: 1e-06 C: 0.01
0.4900  0.4778  0.4778  0.4778  0.4778 - dla: g: 1e-06 C: 1
0.4900  0.4855  0.4909  0.4909  0.4891 - dla: g: 1e-06 C: 100.0
0.4900  0.4973  0.5032  0.5032  0.4932 - dla: g: 1e-06 C: 10000.0
0.4900  0.5009  0.5027  0.4986  0.5014 - dla: g: 1e-06 C: 1000000.0
0.5032  0.5032  0.5032  0.5032  0.5032 - dla: g: 0.0001 C: 1e-06
0.4909  0.4909  0.4909  0.4909  0.4909 - dla: g: 0.0001 C: 0.0001
0.5100  0.5100  0.5100  0.5100  0.5100 - dla: g: 0.0001 C: 0.01
0.4900  0.4868  0.4778  0.4778  0.4905 - dla: g: 0.0001 C: 1
0.4900  0.4973  0.4909  0.4909  0.4918 - dla: g: 0.0001 C: 100.0
0.4900  0.5073  0.5168  0.5023  0.5054 - dla: g: 0.0001 C: 10000.0
0.4900  0.5009  0.4968  0.4896  0.4977 - dla: g: 0.0001 C: 1000000.0
0.5032  0.5032  0.5032  0.5032  0.5032 - dla: 

In [12]:
wypisz(tablicaTickowLogarytmicznaM, 80)

procent = 0.7
Czyste  S     N     S->N    N->S
0.5032  0.5032  0.5032  0.5032  0.5032 - dla: g: 1e-06 C: 1e-06
0.4909  0.4909  0.4909  0.4909  0.4909 - dla: g: 1e-06 C: 0.0001
0.5100  0.5100  0.5100  0.5100  0.5100 - dla: g: 1e-06 C: 0.01
0.4900  0.4778  0.4778  0.4778  0.4778 - dla: g: 1e-06 C: 1
0.4900  0.4909  0.4909  0.4909  0.4909 - dla: g: 1e-06 C: 100.0
0.4900  0.5118  0.5032  0.5032  0.5127 - dla: g: 1e-06 C: 10000.0
0.4900  0.5100  0.5027  0.5073  0.5104 - dla: g: 1e-06 C: 1000000.0
0.5032  0.5032  0.5032  0.5032  0.5032 - dla: g: 0.0001 C: 1e-06
0.4909  0.4909  0.4909  0.4909  0.4909 - dla: g: 0.0001 C: 0.0001
0.5100  0.5100  0.5100  0.5100  0.5100 - dla: g: 0.0001 C: 0.01
0.4900  0.4778  0.4778  0.4778  0.4778 - dla: g: 0.0001 C: 1
0.4900  0.5136  0.4909  0.4909  0.5127 - dla: g: 0.0001 C: 100.0
0.4900  0.4991  0.5027  0.5077  0.5018 - dla: g: 0.0001 C: 10000.0
0.4900  0.5054  0.5100  0.5064  0.5059 - dla: g: 0.0001 C: 1000000.0
0.5032  0.5032  0.5032  0.5032  0.5032 - dla: 

In [13]:
wypisz(tablicaTickowPo1M, 120)

procent = 0.7
Czyste  S     N     S->N    N->S
0.5032  0.5032  0.5032  0.5032  0.5032 - dla: g: 1e-06 C: 1e-06
0.4909  0.4909  0.4909  0.4909  0.4909 - dla: g: 1e-06 C: 0.0001
0.5100  0.5100  0.5100  0.5100  0.5100 - dla: g: 1e-06 C: 0.01
0.4900  0.4778  0.4778  0.4778  0.4778 - dla: g: 1e-06 C: 1
0.4900  0.4968  0.4909  0.4909  0.4955 - dla: g: 1e-06 C: 100.0
0.4900  0.4900  0.5032  0.5032  0.4946 - dla: g: 1e-06 C: 10000.0
0.4900  0.4764  0.4909  0.5032  0.4864 - dla: g: 1e-06 C: 1000000.0
0.5032  0.5032  0.5032  0.5032  0.5032 - dla: g: 0.0001 C: 1e-06
0.4909  0.4909  0.4909  0.4909  0.4909 - dla: g: 0.0001 C: 0.0001
0.5100  0.5100  0.5100  0.5100  0.5100 - dla: g: 0.0001 C: 0.01
0.4900  0.4809  0.4778  0.4778  0.4850 - dla: g: 0.0001 C: 1
0.4900  0.4909  0.4909  0.4909  0.4936 - dla: g: 0.0001 C: 100.0
0.4900  0.4864  0.4909  0.5005  0.4927 - dla: g: 0.0001 C: 10000.0
0.4900  0.5036  0.4873  0.4837  0.5000 - dla: g: 0.0001 C: 1000000.0
0.5032  0.5032  0.5032  0.5032  0.5032 - dla: 

In [14]:
wypisz(tablicaTickowPo1, 120)

procent = 0.7
Czyste  S     N     S->N    N->S
0.5032  0.5032  0.5032  0.5032  0.5032 - dla: g: 1e-06 C: 1e-06
0.4909  0.4909  0.4909  0.4909  0.4909 - dla: g: 1e-06 C: 0.0001
0.5100  0.5100  0.5100  0.5100  0.5100 - dla: g: 1e-06 C: 0.01
0.4900  0.4778  0.4778  0.4778  0.4778 - dla: g: 1e-06 C: 1
0.4900  0.4968  0.4909  0.4909  0.4955 - dla: g: 1e-06 C: 100.0
0.4900  0.4900  0.5032  0.5032  0.4946 - dla: g: 1e-06 C: 10000.0
0.4900  0.4764  0.4909  0.5032  0.4864 - dla: g: 1e-06 C: 1000000.0
0.5032  0.5032  0.5032  0.5032  0.5032 - dla: g: 0.0001 C: 1e-06
0.4909  0.4909  0.4909  0.4909  0.4909 - dla: g: 0.0001 C: 0.0001
0.5100  0.5100  0.5100  0.5100  0.5100 - dla: g: 0.0001 C: 0.01
0.4900  0.4809  0.4778  0.4778  0.4850 - dla: g: 0.0001 C: 1
0.4900  0.4909  0.4909  0.4909  0.4936 - dla: g: 0.0001 C: 100.0
0.4900  0.4864  0.4909  0.5005  0.4927 - dla: g: 0.0001 C: 10000.0
0.4900  0.5036  0.4873  0.4837  0.5000 - dla: g: 0.0001 C: 1000000.0
0.5032  0.5032  0.5032  0.5032  0.5032 - dla: 

In [18]:
imbalance = np.load("imbalance.npy")

In [19]:
wypisz(imbalance)

procent = 0.7
Czyste  S     N     S->N    N->S
0.5032  0.5032  0.5032  0.5032  0.5032 - dla: g: 1e-06 C: 1e-06
0.4909  0.4909  0.4909  0.4909  0.4909 - dla: g: 1e-06 C: 0.0001
0.5100  0.5100  0.5100  0.5100  0.5100 - dla: g: 1e-06 C: 0.01
0.4778  0.4778  0.4778  0.4778  0.4778 - dla: g: 1e-06 C: 1
0.4909  0.4909  0.4909  0.4909  0.4909 - dla: g: 1e-06 C: 100.0
"0.5368  0.5368  0.5354  0.5345  0.5354" - dla: g: 1e-06 C: 10000.0
"0.5413  0.5436  0.5358  0.5345  0.5354" - dla: g: 1e-06 C: 1000000.0
0.5032  0.5032  0.5032  0.5032  0.5032 - dla: g: 0.0001 C: 1e-06
0.4909  0.4909  0.4909  0.4909  0.4909 - dla: g: 0.0001 C: 0.0001
0.5100  0.5100  0.5100  0.5100  0.5100 - dla: g: 0.0001 C: 0.01
0.4778  0.4778  0.4778  0.4778  0.4778 - dla: g: 0.0001 C: 1
"0.5363  0.5368  0.5354  0.5345  0.5358" - dla: g: 0.0001 C: 100.0
"0.5377  0.5386  0.5358  0.5345  0.5358" - dla: g: 0.0001 C: 10000.0
"0.5454  0.5340  0.5358  0.5345  0.5358" - dla: g: 0.0001 C: 1000000.0
0.5032  0.5032  0.5032  0.5032  0.50